In [270]:
import pandas as pd
import numpy as np 
import re
import random
import copy

In [271]:
# The file that includes the slot and intent information.
df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet1")
# The file that includes the types of values inside that slot. 
slot_df = pd.read_excel("Appliance Slot-Intent Manual.xlsx",encoding = "ISO-8859-1", sheet_name="Sheet3")

In [272]:
df['QUESTION_SHOULD_CONTAIN']= df['QUESTION_SHOULD_CONTAIN'].str.lower()
slot_df['Slot']= slot_df['Slot'].str.lower()
df['TARGET_QUESTION']= df['TARGET_QUESTION'].str.lower()
df["TARGET_QUESTION"] = df['TARGET_QUESTION'].str.replace('[\.\?\'\!]','')
## Preprocessing slot_df dataframe
# Slot dataframe trimmed so that the Description is removed. 
slot_df_clean = slot_df.iloc[:,[0,2]]
# replace the Example column with an array of the values separated by a comma
slot_df_clean = slot_df_clean[slot_df_clean['Examples'].notnull()].copy()
slot_df_clean.loc[:,'Examples'] = slot_df_clean.loc[:,"Examples"].apply(lambda x: x.split(', '))
# dictionary of the slots. Key: slot value: possible vallues of the slot. 
slot_dict = dict(zip(slot_df_clean.Slot, slot_df_clean.Examples))
# Split the TARGET_QUESTION column and make it into an array of questions
df.iloc[:,1] = df.iloc[:,1].apply(lambda x: x.split('\n'))
#Temporary extract the TARGET_QUESTION column that was processed above. 
target = df.iloc[:,1]
# A Series of arrays representing the respective slots
slots = df.iloc[:,4].apply(lambda x: x.split(' '))
# The dictionary where key:Intent value:Question
target_dic = df.iloc[:,[0,1]]
# A dictionary of intents key: int value: intent
intent_dict = df.QUESTION_LABEL.to_dict()
slot_dict_boi = copy.deepcopy(slot_dict)
for key,value in slot_dict.items():
    list_slot = []
    for i in value:
        s = i.split()
        for j in range(len(s)):
            if j == 0:
                s[j] = s[j] + ":B-" + key[1:]
            else:
                s[j] = s[j] + ":I-" + key[1:]
        i = ' '.join(s)
        list_slot.append(i)
    slot_dict_boi[key] = list_slot

In [273]:
edge_dict = {}
edge_dict['buy'] = 'buy parts for appliance'
edge_dict['clean'] = 'clean part of a appliance'
edge_dict['code'] = 'error code on appliance screen'
edge_dict['appropriate'] ='appropriate kitchen untensils to use'
edge_dict['remove'] = 'how to remove part from appliance'
edge_dict['replace'] = 'how to replace part from appliance'
edge_dict['cook'] = 'how to cook food on an appliance'
edge_dict['purpose'] = 'the purpose of an appliance feature'
edge_dict['set'] = 'how to set an appliance feature'
edge_dict['use'] = 'how to use an appliance feature'
edge_dict['on_off'] = 'how to turn on/off a feature on an appliance.'
edge_dict['image'] = 'show the image of a part or button in an appliance'
edge_dict['locate'] = 'locate a specific part or button on the appliance'
edge_dict['exist'] = 'Does a certain feature exist?'

In [274]:
edge_to_intent = dict((v, k) for k, v in edge_dict.items())

In [275]:
df = pd.read_excel("Appen\\appliance_utterences_processed_full.xlsx")

In [276]:
df.drop([63],axis=0,inplace=True)

In [277]:
df_trim = df.iloc[:,2:]

In [278]:
df_trim['intent'] = df_trim['edge'].apply(lambda row : edge_to_intent[row])

In [279]:
df_trim

,edge,template,question_should_contain,question,intent
0,buy parts for appliance,Where can I buy a @partType for my @appliance?,@partType @appliance,yes sir Where can I buy a @partType for my @ap...,buy
1,buy parts for appliance,Where can I buy a @partType for my @appliance?,@partType @appliance,for Where can I buy a @partType for my @applia...,buy
2,buy parts for appliance,Where can I buy a @partType for my @appliance?,@partType @appliance,"now, Where can I buy a @partType for my @appli...",buy
3,buy parts for appliance,Where can I buy a @partType for my @appliance?,@partType @appliance,want to bye a @partType and @appliance,buy
4,buy parts for appliance,Where can I buy a @partType for my @appliance?,@partType @appliance,looking for a @partType and @appliance,buy
...,...,...,...,...,...
2788,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,how many @existFeature in the @appliance?,exist
2789,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,when are they going to integrate @existFeature...,exist
2790,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,can you tell me is there an @existFeature in t...,exist
2791,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,please can you tell me is there an @existFeatu...,exist


In [280]:
file_name = 'appliance_data\\appliance_utterances_appen_personal_test.xlsx'
data_current = pd.read_excel(file_name)

In [281]:
df = pd.concat([data_current, df_trim], axis=0)

In [282]:
df

,Unnamed: 0,Unnamed: 0.1,_worker_id,edge,template,question_should_contain,question,intent
0,0.0,18.0,6367365,appropriate kitchen untensils to use,Is it safe to use @cookWare in a @appliance?,@cookware @appliance,will i be able to use @cookware in the @applia...,appropriate
1,1.0,19.0,-,appropriate kitchen untensils to use,-,@cookware @appliance,is the @appliance @cookware safe,appropriate
2,2.0,20.0,-,appropriate kitchen untensils to use,-,@cookware @appliance,is @cookware appropriate to use in the @appliance,appropriate
3,3.0,21.0,-,appropriate kitchen untensils to use,-,@cookware @appliance,is it safe to use @cookware in a @appliance,appropriate
4,4.0,22.0,-,appropriate kitchen untensils to use,-,@cookware @appliance,can i use the @cookware for the @appliance,appropriate
...,...,...,...,...,...,...,...,...
2788,NaN,NaN,NaN,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,how many @existFeature in the @appliance?,exist
2789,NaN,NaN,NaN,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,when are they going to integrate @existFeature...,exist
2790,NaN,NaN,NaN,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,can you tell me is there an @existFeature in t...,exist
2791,NaN,NaN,NaN,Does a certain feature exist?,Is there an @existFeature in the @appliance?,@existFeature @appliance,please can you tell me is there an @existFeatu...,exist


In [283]:
data_trim = df[(df.intent !='exist') & (df.intent != 'locate') & (df.intent != 'image')]

In [284]:
data_trim

,Unnamed: 0,Unnamed: 0.1,_worker_id,edge,template,question_should_contain,question,intent
0,0.0,18.0,6367365,appropriate kitchen untensils to use,Is it safe to use @cookWare in a @appliance?,@cookware @appliance,will i be able to use @cookware in the @applia...,appropriate
1,1.0,19.0,-,appropriate kitchen untensils to use,-,@cookware @appliance,is the @appliance @cookware safe,appropriate
2,2.0,20.0,-,appropriate kitchen untensils to use,-,@cookware @appliance,is @cookware appropriate to use in the @appliance,appropriate
3,3.0,21.0,-,appropriate kitchen untensils to use,-,@cookware @appliance,is it safe to use @cookware in a @appliance,appropriate
4,4.0,22.0,-,appropriate kitchen untensils to use,-,@cookware @appliance,can i use the @cookware for the @appliance,appropriate
...,...,...,...,...,...,...,...,...
2177,NaN,NaN,NaN,how to turn on/off a feature on an appliance.,How do I switch the @onoffFeature off on the @...,@onoffFeature @appliance,I need information about How do I switch the @...,on_off
2178,NaN,NaN,NaN,how to turn on/off a feature on an appliance.,How do I switch the @onoffFeature off on the @...,@onoffFeature @appliance,Can you tell me How do I switch the @onoffFeat...,on_off
2179,NaN,NaN,NaN,how to turn on/off a feature on an appliance.,How do I switch the @onoffFeature off on the @...,@onoffFeature @appliance,how to use @onoffFeature @appliance,on_off
2180,NaN,NaN,NaN,how to turn on/off a feature on an appliance.,How do I switch the @onoffFeature off on the @...,@onoffFeature @appliance,how to enter @onoffFeature @appliance,on_off


In [285]:
data_trim.loc[0,'question_should_contain']

0    @cookware @appliance
0    @partType @appliance
Name: question_should_contain, dtype: object

In [286]:
data_trim.reset_index(drop=True,inplace=True)

In [287]:
data_trim["question"] = data_trim['question'].str.replace('[\.\?\'\,\!\/]','')
data_trim['question']= data_trim['question'].str.lower()
data_trim['question_should_contain']= data_trim['question_should_contain'].str.lower()

C:\Users\l.kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\l.kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\l.kwon\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [301]:
data_trim.to_excel("appliance_data\\appliance_utterances.xlsx")

In [302]:
data_trim = pd.read_excel("appliance_data\\appliance_utterances.xlsx")

In [297]:
def random_slot(slot):
    random_slot = random.randint(0, len(slot_dict[slot])-1)
#     print(slot_dict[slot])
#     print(slot_dict[slot][random_slot])
    return slot_dict_boi[slot][random_slot]

In [298]:
with open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance5\\train','w') as f_train, open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance5\\test','w') as f_test, open('C:\\Users\\l.kwon\\slot_filling_and_intent_detection_of_SLU\\data\\appliance5\\valid','w') as f_valid:
    for i in range(data_trim.shape[0]):
    #    if(i < 30):
    #         print(data_trim.loc[i,'question'])
    #         print(data_trim.loc[i,'question_should_contain'].split(' '))
        for j in range(3):
            slots = data_trim.loc[i,'question_should_contain'].split(' ')
            x = data_trim.loc[i,'question']
            x = re.sub(slots[0], random_slot(slots[0]), x)
            x = re.sub(slots[1], random_slot(slots[1]), x)
            x = x.lstrip()
            x = re.sub(r'((?:^|(?<= ))[a-zA-Z0-9]+(?= |$))', r'\1:O', x) 
            #print(data_trim.loc[i,'intent'])
            x = ' '.join(x.split())
            if((i % 4 == 0) or (i % 4 == 1)):
                f_train.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
                print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
            elif (i % 4 == 2):
                f_test.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
                print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
            else:
                f_valid.write(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
                print(x + " <=> " + data_trim.loc[i,'intent'] + "\n")
f_train.close()
f_test.close()
f_valid.close()

will:O i:O be:O able:O to:O use:O polyester:B-cookware cardboard:I-cookware in:O the:O oven:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O wood:B-cookware in:O the:O microwavawe:B-appliance <=> appropriate

will:O i:O be:O able:O to:O use:O glassware:B-cookware in:O the:O oven:B-appliance <=> appropriate

is:O the:O oven:B-appliance metal:B-cookware trimes:I-cookware safe:O <=> appropriate

is:O the:O microwavawe:B-appliance porcelain:B-cookware safe:O <=> appropriate

is:O the:O oven:B-appliance wicker:B-cookware safe:O <=> appropriate

is:O paper:B-cookware appropriate:O to:O use:O in:O the:O oven:B-appliance <=> appropriate

is:O paper:B-cookware appropriate:O to:O use:O in:O the:O oven:B-appliance <=> appropriate

is:O straw:B-cookware appropriate:O to:O use:O in:O the:O microwavawe:B-appliance <=> appropriate

is:O it:O safe:O to:O use:O aluminum:B-cookware foil:I-cookware in:O a:O microwavawe:B-appliance <=> appropriate

is:O it:O safe:O to:O use:O polyester:B-coo


where:O do:O i:O look:O to:O buy:O a:O screws:B-parttype for:O my:O oven:B-appliance <=> buy

where:O do:O i:O look:O to:O buy:O a:O graphite:B-parttype lubricant:I-parttype for:O my:O microwavawe:B-appliance <=> buy

where:O do:O i:O look:O to:O buy:O a:O door:B-parttype for:O my:O microwavawe:B-appliance <=> buy

please:O tell:O me:O where:O can:O i:O buy:O a:O glass:B-parttype tray:I-parttype for:O my:O oven:B-appliance <=> buy

please:O tell:O me:O where:O can:O i:O buy:O a:O charcoal:B-parttype filter:I-parttype for:O my:O microwavawe:B-appliance <=> buy

please:O tell:O me:O where:O can:O i:O buy:O a:O charcoal:B-parttype filter:I-parttype for:O my:O microwavawe:B-appliance <=> buy

i:O want:O to:O know:O where:O can:O i:O buy:O a:O air:B-parttype fry:I-parttype tray:I-parttype for:O my:O microwavawe:B-appliance <=> buy

i:O want:O to:O know:O where:O can:O i:O buy:O a:O exhaust:B-parttype filter:I-parttype for:O my:O microwavawe:B-appliance <=> buy

i:O want:O to:O know:O where


greeatings:O clean:O part:O of:O a:O appliance:O stainless:B-cleanobject steel:I-cleanobject oven:B-appliance <=> clean

greeatings:O clean:O part:O of:O a:O appliance:O door:B-cleanobject gasket:I-cleanobject microwavawe:B-appliance <=> clean

greeatings:O clean:O part:O of:O a:O appliance:O sugary-spills:B-cleanobject oven:B-appliance <=> clean

hi:O how:O do:O i:O clean:O the:O cooktop:B-cleanobject seal:I-cleanobject on:O the:O microwavawe:B-appliance <=> clean

hi:O how:O do:O i:O clean:O the:O melted-plastic:B-cleanobject on:O the:O oven:B-appliance <=> clean

hi:O how:O do:O i:O clean:O the:O grease:B-cleanobject filter:I-cleanobject on:O the:O oven:B-appliance <=> clean

where:O can:O i:O locate:O them:O cooktop:B-cleanobject oven:B-appliance <=> clean

where:O can:O i:O locate:O them:O glass:B-cleanobject cooktop:I-cleanobject microwavawe:B-appliance <=> clean

where:O can:O i:O locate:O them:O glass:B-cleanobject cooktop:I-cleanobject microwavawe:B-appliance <=> clean

i:O n

is:O it:O safe:O to:O use:O paper:B-cookware bags:I-cookware in:O a:O microwavawe:B-appliance <=> appropriate

is:O it:O safe:O to:O use:O recycled:B-cookware paper:I-cookware in:O a:O microwavawe:B-appliance who:O <=> appropriate

is:O it:O safe:O to:O use:O plastic:B-cookware in:O a:O oven:B-appliance who:O <=> appropriate

is:O it:O safe:O to:O use:O metal:B-cookware in:O a:O microwavawe:B-appliance who:O <=> appropriate

is:O it:O safe:O to:O use:O polyester:B-cookware cup:I-cookware in:O a:O oven:B-appliance why:O <=> appropriate

is:O it:O safe:O to:O use:O aluminum:B-cookware foil:I-cookware in:O a:O oven:B-appliance why:O <=> appropriate

is:O it:O safe:O to:O use:O wicker:B-cookware in:O a:O microwavawe:B-appliance why:O <=> appropriate

you:O can:O use:O the:O pot:O and:O pressure:O thermometer:B-cookware microwavawe:B-appliance <=> appropriate

you:O can:O use:O the:O pot:O and:O pressure:O paper:B-cookware bags:I-cookware oven:B-appliance <=> appropriate

you:O can:O use:O 


what:O was:O how:O do:O i:O remove:O door:B-removeobject from:O the:O oven:B-appliance <=> remove

what:O was:O how:O do:O i:O remove:O lights:B-removeobject from:O the:O microwavawe:B-appliance <=> remove

the:O why:O how:O do:O i:O remove:O door:B-removeobject from:O the:O oven:B-appliance <=> remove

the:O why:O how:O do:O i:O remove:O drawer:B-removeobject from:O the:O microwavawe:B-appliance <=> remove

the:O why:O how:O do:O i:O remove:O rack:B-removeobject from:O the:O oven:B-appliance <=> remove

who:O the:O how:O do:O i:O remove:O lights:B-removeobject from:O the:O microwavawe:B-appliance <=> remove

who:O the:O how:O do:O i:O remove:O rack:B-removeobject from:O the:O microwavawe:B-appliance <=> remove

who:O the:O how:O do:O i:O remove:O lights:B-removeobject from:O the:O oven:B-appliance <=> remove

what:O the:O how:O do:O i:O remove:O rack:B-removeobject from:O the:O oven:B-appliance <=> remove

what:O the:O how:O do:O i:O remove:O stains:B-removeobject from:O the:O oven:B


what:O must:O i:O do:O to:O cook:O frozen:B-foodtype fish:I-foodtype fingers:I-foodtype with:O the:O oven:B-appliance really:O <=> cook

what:O must:O i:O do:O to:O cook:O fresh:B-foodtype chicken:I-foodtype wings:I-foodtype with:O the:O oven:B-appliance really:O <=> cook

how:O can:O i:O cook:O lamb:B-foodtype chops:I-foodtype with:O the:O microwavawe:B-appliance <=> cook

how:O can:O i:O cook:O fresh:B-foodtype chicken:I-foodtype wings:I-foodtype with:O the:O microwavawe:B-appliance <=> cook

how:O can:O i:O cook:O breaded:B-foodtype chicken:I-foodtype breast:I-foodtype with:O the:O oven:B-appliance <=> cook

how:O can:O i:O easily:O cook:O breaded:B-foodtype cauliflower:I-foodtype with:O the:O oven:B-appliance <=> cook

how:O can:O i:O easily:O cook:O breaded:B-foodtype chicken:I-foodtype breast:I-foodtype with:O the:O microwavawe:B-appliance <=> cook

how:O can:O i:O easily:O cook:O fresh:B-foodtype vegetables:I-foodtype with:O the:O oven:B-appliance <=> cook

how:O can:O i:O safe

which:O function:O is:O better:O convention:B-purposefeature bake:I-purposefeature on:O the:O microwavawe:B-appliance <=> purpose

which:O function:O is:O better:O self:B-purposefeature clean:I-purposefeature on:O the:O oven:B-appliance <=> purpose

could:O someone:O please:O tell:O me:O what:O is:O the:O purpose:O of:O smart:B-purposefeature control:I-purposefeature in:O my:O microwavawe:B-appliance <=> purpose

could:O someone:O please:O tell:O me:O what:O is:O the:O purpose:O of:O delay:B-purposefeature start:I-purposefeature in:O my:O microwavawe:B-appliance <=> purpose

could:O someone:O please:O tell:O me:O what:O is:O the:O purpose:O of:O hot:B-purposefeature surface:I-purposefeature indicator:I-purposefeature in:O my:O microwavawe:B-appliance <=> purpose

how:O can:O i:O know:O what:O function:O the:O delay:B-purposefeature start:I-purposefeature has:O in:O an:O oven:B-appliance <=> purpose

how:O can:O i:O know:O what:O function:O the:O lamp:B-purposefeature has:O in:O an:O ov


how:O do:O i:O start:O my:O add:B-usefeature 30:I-usefeature sec:I-usefeature in:O the:O microwavawe:B-appliance <=> use

how:O do:O i:O start:O my:O child:B-usefeature lock:I-usefeature in:O the:O microwavawe:B-appliance <=> use

how:O do:O i:O start:O my:O 12:B-usefeature hour:I-usefeature energy:I-usefeature saving:I-usefeature in:O the:O oven:B-appliance <=> use

how:O to:O use:O my:O dehydrate:B-usefeature oven:B-appliance <=> use

how:O to:O use:O my:O kitchen:B-usefeature timer:I-usefeature button:I-usefeature oven:B-appliance <=> use

how:O to:O use:O my:O soften:B-usefeature melt:I-usefeature oven:B-appliance <=> use

how:O to:O set:O my:O temp:B-usefeature adjust:I-usefeature microwavawe:B-appliance <=> use

how:O to:O set:O my:O eco:B-usefeature mode:I-usefeature microwavawe:B-appliance <=> use

how:O to:O set:O my:O soften:B-usefeature melt:I-usefeature microwavawe:B-appliance <=> use

how:O to:O make:O my:O reheat:B-usefeature oven:B-appliance <=> use

how:O to:O make:O m


could:O you:O check:O the:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature in:O the:O microwavawe:B-appliance <=> on_off

could:O you:O check:O the:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature in:O the:O microwavawe:B-appliance <=> on_off

could:O you:O check:O the:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature in:O the:O oven:B-appliance <=> on_off

do:O you:O need:O the:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature in:O the:O oven:B-appliance <=> on_off

do:O you:O need:O the:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature in:O the:O oven:B-appliance <=> on_off

do:O you:O need:O the:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature in:O the:O oven:B-appliance <=> on_off

how:O do:O i:O switch:O the:O self:B-onofffeature cleaning:I-onofffeature cycle:I-onofffeature off:O on:O the:O oven:B-appliance wen:O <=> on_off

how:O do:O i:O switch:O the:O wifi:B-onofffeature of